# Development 


## <span style="color:red">Name: Primary Task</span>   
**Primary Task:** One of *Image to Value(s)*, *Image to Image*, or *2D to 3D*  </span>  

### <span style="color:red">Name: Primary Focus</span>      
<span style="color:red">**Primary Focus:** Describe the main goal of your use case, such as *segmentation* or *object counting*. </span>  

### <span style="color:red">Name: Application</span>  
<span style="color:red">**Application:** Specify the implemented exemplary application for your use case, e.g., *segmentation of cellular structures*.  </span>  

#### <span style="color:red">Name: Challenge</span>    
<span style="color:red">**Challenge:** If your use case addresses a specific challenge (e.g., explainability, handling small datasets), mention it here. </span>   

#### <span style="color:red">Required Labels</span>  
<span style="color:red">**Required Labels:** List the necessary labels for training your model (e.g., *segmentation mask*, *location*).  </span>  

---  

## TL;DR 🧬✨
<span style="color:red">Add a short description of your use case here.</span>    

You can also include a teaser image like this:  
`![Teaser](./images/Teaser.png)`  

---



This notebook allows you to do: 

- ✅ **Model Training**: Model training involves optimizing a neural network on a given dataset to learn meaningful patterns, resulting in a trained model that can be used for Evaluation or Inference.  

    - ✅ **Hyperparameter Search**: Before full model training, it is recommended to perform a hyperparameter search. This involves multiple short training runs on different hyperparameter sets to approximate full training performance. The best-performing set is then selected. Expanding the search space or using better approximations can improve results but requires more time and compute resources. Make sure you have enough Lightning AI credits before doing so.  

    - ✅ **Model Training + Validation**: Once the best hyperparameters are selected, the model is trained on the dataset while monitoring performance on a validation set. This ensures that the model generalizes well and helps prevent overfitting.  

- ✅ **Evaluation**: Evaluation assesses the performance of a trained model. This can be done on the test split of the training dataset (this should usually be done after training the model) or on a completely new dataset to check generalization. To evaluate on the full dataset, enable the "evaluate on full" checkbox before running the evaluation cell, only do this if you did not train the model on the same data.  

---


# Setup and Imports

In [1]:
# auto reload imports
%load_ext autoreload
%autoreload 2

# imports from the template 
from deepEM.Utils import create_text_widget, print_info, find_file, create_checkbox_widget, set_seed
from deepEM.Logger import Logger
from deepEM.ModelTuner import ModelTuner

# costum implementation
from src.ModelTrainer import ModelTrainer


# import all required libraries
import os
set_seed(42)

# 1. Data

## 1.1 Data Acquisition  

<span style="color:red">
Provide a brief description of the exemplary data used in your use case.  
If necessary, include guidelines or tips for collecting custom datasets.  </span>


## 1.2. Data Anntation

> **Hint**: Note that this step is only required if you wish do adapt this use case to your own dataset. If you like to use the provided data, this step can be skipped. 

<span style="color:red">
Include a step-by-step guide on how to annotate data such that it can be used in a plug and play fashion. This should allow EM researchers for example to exchange the segmentation of cellular structures with the segmentation of virus particles to match their labratories needs. 

For data annotation we recomment using the <a href="https://www.cvat.ai/">CVAT</a> (Computer Vision Annotation Tool) tool. For further instructions, we refer to our <a href="">Getting Started</a> page.

Example step-by-step guide is shown below for the case of annotating location annotations. </span>


### 1.2.1. Create a New Task

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-1.png" alt="Create New Task1" width="500">
  <br>
</p>
When starting CVAT, you first need to create a new task. You can give it a name, add annotation types and upload your data.

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-2.png" alt="Create New Task2" width="500">
  <br>
</p>

Next, click on the `Add label` button. Name it based on the class you want to annotate. In our case one of *"naked", "budding", "enveloped"*. As annotation type choose `Points`. You should also pick a color, as this will simplify the annotation process. For adding new class click `Continue`. Once you added all nessecary classes click `Cancel`. 


<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-3.png" alt="Create New Task3" width="500">
  <br>
</p>

Now you can upload the data you wish to annotate. Finally, click `Submit & Open` to continue with the annotation of the uploaded data. 

### 1.2.2. Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-4.png" alt="Annotate1" width="500">
  <br>
</p>

This will open following view. Click on the job (in this view the `Job #1150022`) to start the annotation job. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-5.png" alt="Annotate2" width="500">
  <br>
</p>

To then annotate your data, select the `Draw new points` tool. Select the Label you wish to annotate from the dropdown menue. Then click `Shape` to annotate individual virus capsids with the label class. (Track will allow you to place annotations over multiple frames, which is helpful when annotating videos, tomograms or similar). You can use the arrows on the top middle to navigate through all of your data and to see your annotation progress. 

### 1.2.3. Save Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-6.png" alt="Save1" width="500">
  <br>
</p>

Once you are done annotating data, click on the `Menu` and select `Export job dataset`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-7.png" alt="Save2" width="500">
  <br>
</p>

During export select the `CVAT for Images 1.1` format and give the folder a name. It will prepare the dataset for download. If you have the annotated images stored locally, there is no need to enable `Save Images`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-8.png" alt="Save3" width="500">
  <br>
</p>

In the horizontal menu bar at the top go to `Requests`. It will show a request Export Annotations. On the right of this request click on the three dots on the right to download your exported, annotated data. This will download a .zip file containing the annotation file in .xml format. The name of the file should be "annotations.xml".



## 1.3. Data Preprocessing

> **Hint**: Note that this step is only required if you wish do adapt this use case to your own dataset. If you like to use the provided data, this step can be skipped. 

<span style="color:red">Include a step-by-step guide for preprocessing the data based on common EM data formats like `.tif`to match your notebooks needs.</span>

<span style="color:red">[ImageJ](https://imagej.net/ij/) can be a helpful software in this case, as it is an open source, Java-based image processing software that runs on multiple platforms and offers a wide range of features, including automation with macros, extensive community support, and a large library of tools and plugins.</span>

<span style="color:red">In the following we showcase an example usecase to import and export data into required formats. </span>





<p align="center">
  <img src="images/ImageJ-1.png" alt="ImageJ1" width="500">
  <br>
</p>

<span style="color:green">This tool allows to `import` a large amount of different, commonly used file formats im EM. </span>

<p align="center">
  <img src="images/ImageJ-2.png" alt="ImageJ2" width="500">
  <br>
</p>

<span style="color:green">Using the provided `Save As..` functionality allows to save the imported files as a `Image Sequence` in .tif format or single `.tif` files.</span>

## 1.4. Data Structuring

<span style="color:red">Your use case will require the data to be in a specific structure, such that it can be used for  training. Describe this structure here. </span>

<span style="color:green">**An example description could look like this:** </span>

<span style="color:green">The provided notebook requires that all training, validation and testing data is placed within a single folder. Splitting the data into train, test and validation will be done during runtime. </span>

<span style="color:green">Additionally, the generated `annotations.xml` should be put in the same folder as the .tif images.</span>

<span style="color:green">You can check the exemplary data provided at `data/tem-herpes/` for clarification.</span>

<span style="color:green">An example with five images and the corresponding annotation is shown below: </span>

```
/data/tem-herpes/
├── image_001.tif
├── image_002.tif
├── image_003.tif
├── image_004.tif
├── image_005.tif
└── annotations.xml

```

> *Execute the cell below to show a text form. Within this text form you need to define the path to your training data (i.e. `data/`).*


In [ ]:
data_widget = create_text_widget("Data Path:","./data","Enter the path to your data folder.")
display(*data_widget)

> *Execute the cell below to set and check the provided Data Path from the text form above.*

In [ ]:
data_path = data_widget[0].value
print(f"[INFO]::Data path was set to: {data_path}")

# 2. Model Training

## 2.1. Setup Logging

By executing the cell below, we setup the logging directory for the hyperparameter search, model training and evaluation. 
The logger creates a folder at `./logs/<datafoldername>-<currentdatetime>/`. 

For each training run there will be one subfolder within the log directory. Training runs of hyperparameter sweeps are called `Sweep_<idx>`, while the subfolder of the final training run is called `TrainingRun`. During evaluation there will be one more subfolder created called `Evaluate`. 

Within each subfolder folder there will be logging of: 

- the used hyperparameters, (`<log-path>/<subfolder>/hyperparameters.json`)
- the best performing model checkpoint based on the validation loss (`<log-path>/<subfolder>/checkpoints/best_model.pth`)
- the last model checkpoint (`<log-path>/<subfolder>/checkpoints/latest_model.pth`)
- visualizations of training and validation curves (`<log-path>/<subfolder>/plots/training_curves.png`)
- qualitative visualization of sampled validation images (`<log-path>/<subfolder>/samples/`)
- results on test metrics (`<log-path>/<subfolder>/test_results.txt`)
- qualitative visualization of sampled test images (`<log-path>/<subfolder>/samples/`)


<span style="color:red">Add a short description of what the sample visualizations show in your use case.</span>

Visualization of training and validation curves will also be shown after every successful training run within this notebook. They can help to identify possible issues like overfitting during training. For more details, we refer to [this guide](https://www.kaggle.com/code/ryanholbrook/overfitting-and-underfitting).

> *Exectue the cell below to setup the logger. **Hint** If you wish to train a new model, you can reexecute this cell, to generate a new log directory - allowing you to now override the previousely trained model.*




In [ ]:
logger = Logger(data_path)

## 2.2. Hyperparameter Tuning

Hyperparameters in deep learning control how a model is trained. Unlike learned model parameters, they are set before training. Hyperparameter tuning explores different configurations by training the model multiple times and selecting the best-performing settings based on validation performance. Since this process is time- and resource-intensive, training runs are often limited in duration or dataset size.

Our playground automates hyperparameter search using grid search, testing all possible combinations of selected hyperparameters. The search space is initially defined by deep learning (DL) experts, who also provide explanations so electron microscopy (EM) specialists can refine it as needed. In order to do so, you can adapt the form below. Each sweep hyperparameter should be separated by `,`. Floating point values should be written like `0.1`. 
Logging estimates the remaining time for individual runs and the full sweep, though early estimates may be inaccurate.

While not strictly required, a hyperparameter search should be performed at least once per dataset to ensure optimal model performance. Interrupting the search early may yield suboptimal results. The automatic sweep stores tested configurations and the best-performing parameters in the training data directory under `Sweep_Parameters`, allowing reuse for future training, or to resume the sweep if the kernel got interrupted.


> *Execute the cell below to show the form of the hyperparameter search space. **Hint** If you've changed parameters and want to reset them to the defaults, reexecute the cell.*

In [ ]:
# hyperparameter search
model_trainer = ModelTrainer(data_path, logger)

hyperparameter_tuner = ModelTuner(model_trainer, data_path, logger)
form = hyperparameter_tuner.create_hyperparameter_widgets()
display(form)


> *If you wish to run a hyperparameter sweep based on the parameters above, please execute the cell below. This should be done at least once per dataset. Note that this can take a while.*

In [ ]:
best_config = None
hyperparameter_tuner.update_config(form)
best_config = hyperparameter_tuner.tune()

Our automatic hyperparameter tuning is able to find the best performing set of hyperparameters based on the setting shown above. 

However, there can be scenarios, where additional flexibility is required. Therefore, you are able to change these hyperparameters in the following. 

> *Execute the cell below to show and possibly adapt the currently chosen hyperparameters.*

In [ ]:
form = hyperparameter_tuner.edit_hyperparameters()
display(form)

Additionally, you can increase the number of training epochs. An 'epoch' in deep learning is one full pass of the model through all the training data, where the model learns and adjusts to improve its predictions. Higher number of epochs leads to longer training but can further improve model performance.

> *Execute the cell below to show and possibly adapt the number of training epochs. Leave as is, if you want to train with the suggestion of the DL expert.*

In [ ]:
model_trainer.reduce_epochs = None
model_trainer.set_epochs()
epochs_widget = create_text_widget("Epochs:",str(model_trainer.num_epochs),"Higher number of epochs leads to longer training but can further improve model performance.")
display(*epochs_widget)

> *Execute the cell below to set the hyperparameters and number of training epochs for your training run, based on the forms above.*

In [ ]:
best_config = hyperparameter_tuner.update_hyperparameters(form)
model_trainer.num_epochs = int(epochs_widget[0].value)
print_info(f"Will use following hyperparameters for future training: {best_config} with number of epochs: {model_trainer.num_epochs}.")

## 2.3. Training and Validation

In this section we train and validate the model based on the provided data and hyperparameters resulting from the previous sweep.

Training in deep learning is the process where a model learns patterns from labeled data (the one provided at the top of this notebook) by optimizing its parameters through backpropagation. 
Validation involves using a separate dataset to evaluate the model's performance during training, ensuring it generalizes well to unseen data.

Training and validating a model can take a lot of time (ranging from minutes to hours, days or even weeks) depending on the model, the training procedure and the dataset. Our logging module provides approximate times for training, which you can see below the executed training cell or at the `log.txt` within the current log directory (i.e. `<log-dir>/TrainingRun/`). However, these times can be inaccurate, especially at the beginning of training. 

### Model Checkpoint
In the following you can provide a model checkpoint for training. There are two different scenarios when you might want to provide a model checkpoint:

1. You wish to resume training. This means, training will picks up exactly where it left off, including learned patterns and settings. This is useful if training was interrupted and needs to be finished from the last saved state. To do so, you need to provide a model checkpoint in the text form below. You can find the last saved checkpoint inside the runs logging directory (`<log-dir>/TrainingRun/checkpoints/latest_model.pth`).
2. If you wish to finetune your model. Fine-tuning starts training from the beginning (epoch 0) but uses a pre-trained model as a starting point, already having knowledge about some previousely learned patterns, to improve its performance on a new task or dataset. You can find the best model checkpoint inside the runs logging directory (`<log-dir>/TrainingRun/checkpoints/best_model.pth`).

If you wish to finetune the model, you need to check the checkbox below. If you only provide the path to a directory, it will look for a `best_model.pth` or `latest_model.pth` accordingly, within this directory.


If you want to start training from scratch (which is usually the case), you can leave the text form below empty.

> *Execute the cell below to show a text form. If you wish to resume training, or do finetuning you need to provide a path to a model checkpoint. Leave it empty for standard training.*

In [ ]:
resume_widget = create_text_widget("Model Checkpoint Path:","","If you wish to resume an earlier training, or do finetuning, enter the path to the latest_model.pth or the best_model.pth file here.")
checkbox_widget, description_widget = create_checkbox_widget(
    name="Enable Finetuning",
    value=False,
    description="Check this box to do finetuning based on the provided model checkpoint above. This means, the models weights will only be used for initializing the model, training will be then done as usual."
)
display(*resume_widget, checkbox_widget, description_widget)

In [ ]:
resume_training = resume_widget[0].value
finetuning = checkbox_widget.value
if(resume_training):
    if(os.path.isfile(resume_training)):
        logger.log_info(f"Found model checkpoint at {resume_training}.")

    else: 
        if(not finetuning):
            resume_training = find_file(resume_training, "latest_model.pth")           
        else: 
            resume_training = find_file(resume_training, "best_model.pth")
            
            
        if(resume_training is None):
            logger.log_error(f"Could not find resume path at {resume_widget[0].value}. Will start training from scatch.")
        else: 
            logger.log_info(f"Found model checkpoint at {resume_training}.")

else:
    resume_training = None
logger.init("TrainingRun")
model_trainer.resume_from_checkpoint = resume_training
model_trainer.finetuning = finetuning
model_trainer.prepare(best_config, num_epochs = int(epochs_widget[0].value))


> *Execute the cell below if you wish to **train** a model. Note that this can take a while.* 

In [ ]:
model_trainer.fit()

# 3. Model Evaluation
Evaluation in deep learning is the process of evaluating a trained model on a separate, unseen dataset to measure its final performance. It provides an unbiased assessment of the model's ability to generalize to new data.

## 3.1. Choose Model 

In this section we choose the model for testing. 
If you leave the `Model Path` empty in the text form below, it will use the last model trained.
Otherwise, you can define the path to the models best weights at `<log-path>/TrainingRun/checkpoints/best_model.pth` or by providing a path to a directory, which contains `best_model.pth` (like `<log-path>/TrainingRun/`). This allows you to also test shared models or previousely trained models.

> *Execute the cell below to show the text form for selecting a model for testing. Leave empty to test the previouely trained model.*

In [ ]:
model_widget = create_text_widget("Model Path:","","If you wish to test a specific model, you can here define the path to its checkpoint (for example: logs/tem-herpes_2025-02-03_11-42-43/TrainingRun/checkpoints). Leave empty to test the last trained model.")
display(*model_widget)


You can check the generalizability of your model, or a model you've received from a fellow collegue by testing it on another set of annotated data. In this case, you probably want to use all data within your provided data path for testing. If this is the case, you can check the checkbox below to do so. Note that this should only be done if you didn't train the model on the same data, but you'd like to assess its performance on a different dataset, to test its generalizability. For more details check the "Whats Next? - Satisfied?" section at the bottom of the notebook. 

> *Execute the cell below to show a checkbox. When checking the box, you will evaluate the model on the full data provided at the data directory. Only do this, if you did not train the model on the same data.*

In [ ]:
checkbox_widget_test_split, description_widget_test_split = create_checkbox_widget(
    name="Evaluate on full dataset",
    value=False,
    description=f"Check this box to use all the data provided in '{data_path}' for evaluation. Only do this, if you did not train the model on the same data."
)
display(checkbox_widget_test_split, description_widget_test_split)

## 3.2. Evaluate
We finally evaluate the provided model on the test set. We investigate following metrics: 

<span style="color:red"> Add a description of the used metrics here. </span>

<span style="color:green">

- **Mean Squared Error (MSE)**: A distance metric to compute the squared difference between the prediction and the label. In this use case this means, we compute the difference between the real EM tilt images with the computed EM images based on the learned reconstruction. If MSE is low, the model is able to correctly represent the underlying sample within the tomogram. The lower the metric, the better. 

- **MSE phantom**: The first metric is not able to capture generalizability of the model to unknown tilt angles, as it only computes the MSE on the tilt series which was used for training. But as we are using synthetic data, we do have access to the underlying phantom volume (the synthetic sample used to compute the tilt series). We hence compare the learned sample with this phantom volume, again by computing the MSE. 

#### **Summary**
| Metric  | Meaning | Interpretation |
|---------|---------|---------|
| **MSE**  | Distance between true tilt series and tilt series based on the learned reconstruction. | Lower is better |
| **MSE phantom**  | Distance between the phantom volume (the synthetic sample used to compute the tilt series) and the learned sample (tomogram) | Lower is better | 

</span>


<span style="color:red">Add a description of your visualizations here.</span>




These visualizations are saved to `<log-path>/Evaluate/samples/test_*`. 

> *If you wish to evaluate a model (recommended), execute the cell below.*

In [ ]:
from pathlib import Path 
start_evaluation = False
eval_model = model_widget[0].value
if(eval_model):
    eval_model = Path(eval_model)
    if(eval_model.is_dir()):
        eval_model = Path(find_file(eval_model, "best_model.pth")) 
    if(not eval_model.is_file()):
        logger.log_error(f"Could not find model at {eval_model}. Make sure to train a model before evaluation.")
        eval_model = None
    else: 
        start_evaluation = True
else:
    recent_logs = logger.get_most_recent_logs()
    eval_model = ""
    for dataname, log_path in recent_logs.items():
        if(dataname == Path(data_path).stem):
            eval_model = Path(log_path+"/TrainingRun/checkpoints/best_model.pth")
            if(not eval_model.is_file()):
                logger.log_error(f"Could not find a trained model at {eval_model}. Make sure you fully train a model first before evaluating.")
            else:
                logger.log_info(f"Found most recent log at {eval_model}")
                start_evaluation = True
        else: 
            continue
    if(not start_evaluation):
        logger.log_error("Could not find a trained model. Make sure you train a model first before evaluating.")
      
if(start_evaluation):
    evaluate_on_full = checkbox_widget_test_split.value
    model_trainer.logger.init(f"Evaluate")
    model_trainer.logger.init_directories()
    model_trainer.load_checkpoint(eval_model)
    model_trainer.test(evaluate_on_full)      

# Whats Next?

## Not satisfied? 
If you are not satisfied with the evaluation performance of your model you have multiple options: 
1. Extend the hyperparameter search by adding more, different hyperparameters - they might work better.
2. Extend the hyperparameter search to a larger data subset or longer training. This will lead to more accurate results of the sweep, but it will require more compute time. If you do so, make sure to delete the previous state of your hyperparameter search by deleting (or renaming) the folder `<data-path>/Sweep_Parameters`. 
3. Run multiple training runs with different custom parameters (you can define them right before the cell for training your model).
4. Extend the number of training epochs. Longer training can lead to better performance of the model. To do so, you can again define the number of epochs in the cell right before training the model.
5. Finetune or train the model on your own annotated dataset. To do so, we recomment using CVAT for annotation. A general guide for annotating your data with CVAT can ge found on our ["Getting Started - 4. Data Annotation"](https://viscom-ulm.github.io/DeepEM/getting-started.html). A more specific guide for this use case can be found at the top of this notebook. 
6. Check the training and sweep runs loggings and train/val curves - maybe you found an issue with the training itself?
 
## Satisfied?
If you are satisfied with the results of your trained or evaluated model, there are multiple things to test next:

1. Check the generalizability of your trained model. To do so, you can evaluate your model on different, annotated datasets. Upload your annotated data and define it as `data_path` at the top of this notebook. Check the top of the notebook for data structuring and annotation formats. Run all cells except for `hyperparameter tuning`  and `training`. Then, check the checkbox "evaluate on full dataset" such that all of your uploaded data is considered for evalutation. 
2. Use the trained model for inference. That means using the model on unseen, unlabeled data for the support of your EM data analysis. To do so, open the `2_Inference.ipynb` and follow the steps provided.
3. Additionally, you can share your training code and model weights with other collegues. An easy way on how to do this can be found on our website under ["Getting Started - 5. Collaboration"](https://viscom-ulm.github.io/DeepEM/getting-started.html).
